# Introducción

Desarrollo de un modelo de Machine Learning para la identificación automática del hablante.

Se Creó una red Perceptrón Multi Capa, modelando lo planteado en el paper: Identificación automática del hablante mediante redes neuronales de la UNER

Link: https://www.researchgate.net/publication/265964745_Identificacion_Automatica_del_Hablante_mediante_Redes_Neuronales
        
Torres, Humberto & Rufiner, Hugo. (1999). Identificación Automática del Hablante mediante Redes Neuronales.

# 

# 

# Imports

In [ ]:
!pip install numpy==1.20.0

In [ ]:
import scikitplot as skplt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import librosa
from librosa import display
from joblib import load, dump
from datetime import datetime
import math
from google.cloud import storage
from google.oauth2 import service_account
from project_lib import Project
import json
import warnings
import pickle

In [ ]:
!pip install -Iv scikit-learn==1.0

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn import metrics

In [ ]:
from google.cloud import storage
from google.oauth2 import service_account

# 

In [ ]:
#Configuración general de visualización del notebook

#Seteo para ver todas las columnas cuando muestro un dataframe
pd.set_option('display.max_columns', None)

#Seteo para que no muestre los Warnings de valores asignados en un slice de una copia de un DF. (Simplifica lectura de Logs)
pd.options.mode.chained_assignment = None  # default='warn'

warnings.filterwarnings("ignore")

In [ ]:
# ME CONECTO AL PROYECTO PARA PODER HACER LA LECTURA DE LOS ARCHIVOS QUE TENGO GURDADOS DENTRO

#Token Entorno
project = Project(project_id='821deb35-d3d9-4a8d-a5e7-b1823688b22f', 
                  project_access_token='ACCESS_TOKEN')

# 

# 

# Variables Globales

In [ ]:
#Cantidad de personas usadas en el entrenamiento
cant_personas_train = 8

#Cantidad de Coeficientes MFCC a extraer(incluidos DELTAS)
cant_MFCC = 32

#Nombre del archivo donde se guarda el usuario a entrenar
user_to_train_filename = "user_to_train.txt"


# 

# 

# DEFINICIÓN DE FUNCIONES

In [ ]:
#Notar que por default, se establecen los valores:

#sr=16000. Tasa de muestreo de los audios de TIMIT

#n_mfcc=16. Cantidad de Coeficientes utilizados en el Paper a replicar

#n_fft=512. Tamaño de la ventana en la transformada de Fourier. (Notar que, sin solapamiento,
#se obtendrán 16000/512 frames por segundo)

#hop_length=256. Tamaño del salto. Con 256, se logra un solapamiento del 0.5 entre cada ventana de la FFT, logrando así
#captar características propias de los pronunciamientos y fonemas.



def loadMfccArchivo(archivoVoz, sr=16000, n_mfcc=16, n_fft=512, hop_length=256):
    """
    Carga de los coeficientes Cepstrum en la escal Mel de un archivo utilizando la librería librosa.
    
    
    
    Params:
    
    archivoVoz = archivo a procesar
    sr = Sampling Rate del archivo (en Hz)
    n_mfcc = cantidad de coeficientes cepstrales a extraer
    n_fft = tamaño de la ventana de la transformada de Fourier
    hop_length = tamaño del salto para el procesamiento en frames
    
    
    
    Para mas información:
    
    MFCC = https://es.wikipedia.org/wiki/MFCC
    
    Librosa MFCC = https://librosa.org/doc/main/generated/librosa.feature.mfcc.html
    """
    y,sr=librosa.load(archivoVoz, sr=sr)
    return librosa.feature.mfcc(y, sr=sr, n_mfcc=n_mfcc, dct_type=2, norm='ortho',
                                hop_length=hop_length, n_fft=n_fft) 

In [ ]:
#Funcion para agregar en una matriz, un vector de identificacion.
#El vector identifica, para el i-ésimo vector de la matriz de mfcc, a que persona corresponde dicho MFCC.

#Agrega a la matriz un vector de longitud K, donde K son la cant. personas a identificar, 
#siendo todas las posiciones 0, salvo la persona a la que corresponde el frame que es 1.

#I.E. Siendo K=5 personas a identificar, la persona 3 identificada en el frame, se agregará al vectorIdentificados
#el frame actual, y a personaIdentificada el array (0,0,1,0,0)

#A efectos de replicar el paper, se establece como default la cantidad de personas en 8, en caso de testear con menos
#se deja a libre elección el parámetro


def agregarIdentificacionPersona(personaIdentificada, vectorIdentificados, cantPersonas=8):
    
    """
    Agregado de la identificación de una persona a un vector de personas identificadas, y del frame actual
    a un vector con frames de audios
    
    Agregado en el final, para no perder referencia de que el n-ésimo frame del vector de frames identificados
    corresponde a la persona marcada con 1 en el n-ésimo frame del vector de personas identificadas
    
    Params:
    
    personaIdentificada = número de la persona identificada
    vectorIdentificados = vector de personas identificadas hasta el momento
    cantPersonas = cantidad de personas a analizar pertenencia


    """
    
    #Creo un vector de todos zeros. Luego, con índice comenzando en 0, marco a la persona identificada con 1
    #apilo verticalmente al final, y retorno el vector con la persona nueva identificada
    
    vectorCeros = np.zeros((cantPersonas,))              
    vectorCeros[personaIdentificada-1] = 1
    vectorIdentificados = np.vstack((vectorIdentificados, vectorCeros))
    return vectorIdentificados

In [ ]:
def guardar_output_como_pickle(filename, model):
    
    
    with open(filename, 'wb+') as z:
        
        data = pickle.dumps(model)

        project.save_data(filename, data, set_project_asset=True, overwrite=True)

In [ ]:
def cargar_modelo_pickle(filename):
    
    modelo = pickle.load(project.get_file(filename))
    
    return modelo
    

# 

# Usuario a entrenar

In [ ]:
#ASI SE CARGA UN TXT
user_to_train = project.get_file(user_to_train_filename).read().decode('UTF-8').upper()
print("Se entrenará el modelo para "+user_to_train)

# 

# Carga de audios de firebase

Desde Firebase se leen los audios y se separan entre 8 personas aleatorias con las que se entrenará el modelo, y las que el modelo NO conoce (impostores).


Con todas ellas luego se evaluará la performance del modelo

In [ ]:
#Creamos las credenciales con las que nos conectamos a la cuenta de servicio para obtener los archivos
#Creamos el cliente del storage para luego utilizar las funcionalidades
#obtenemos los buckets y los imprimimos para ver que no haya cambios

credentials_file_name = "cuidartech-7e54f-firebase-adminsdk-8qlnh-6b3bfdfa23.json"

json_credentials = json.load(project.get_file(credentials_file_name))

credentials = service_account.Credentials.from_service_account_info(json_credentials)

storage_client = storage.Client(credentials=credentials)

buckets = list(storage_client.list_buckets())
print(buckets)

In [ ]:
#Creamos la instancia del bucket en donde tenemos los archivos y luego listamos todos los blobs

bucket = storage_client.get_bucket("cuidartech-7e54f.appspot.com")

blobs = list(bucket.list_blobs(prefix="audios/"))


In [ ]:
blobs

In [ ]:
#Nos quedamos con todos los usuarios que tenemos audios de ellos, y sus nombres únicos.
#Ordenamos aleatoriamente para entrenar al modelo con personas aleatorias
#Una vez ordenado, nos quedamos con los primeros 8

users = [ blob.name.split('/')[1].upper() for blob in blobs]

users = np.unique(users)

np.random.shuffle(users)

users = users.tolist()

users

In [ ]:
#Creamos los usuarios con los que se entrenará el modelo.
#En primer lugar eliminamos al usuario que queremos entrenar de la lista de todos los usuarios, agregamos 7 usuarios al azar de
#esta lista para entrenar al modelo, y agregamos como octavo al que se quiere identificar, es decir, para el que voy a entrenar.


users.remove(user_to_train)


used_users = users[:7]

used_users.append(user_to_train)


np.random.shuffle(used_users)


used_users

In [ ]:
#Nos quedamos con los usuarios no utilizados para probar impostores

not_used_users = users[7:]
not_used_users

In [ ]:
#Creamos los 3 diccionarios a usar. Uno con todos los audios y luego los de train y tests.

diccionario_audios_used_users = dict.fromkeys(used_users, [])

diccionario_audios_train = dict.fromkeys(used_users, [])

diccionario_audios_test = dict.fromkeys(used_users, [])



In [ ]:
#Creamos el diccionario de los audios de los impostores

diccionario_audios_not_used_users = dict.fromkeys(not_used_users, [])


In [ ]:
#Limpiamos los diccionarios para que cada key haga referencia a un array diferente

for user_to_clean in diccionario_audios_used_users.keys():
    diccionario_audios_used_users[user_to_clean] = []
    diccionario_audios_train[user_to_clean] = []
    diccionario_audios_test[user_to_clean] = []

In [ ]:
#Limpiamos los diccionarios para que cada key haga referencia a un array diferente

for user_to_clean in diccionario_audios_not_used_users.keys():
    diccionario_audios_not_used_users[user_to_clean] = []

In [ ]:
#De todos los blobs que tenemos cargamos los nombres de blobs de los usuarios que actualmente vamos a entrenar.
#Una vez que los tenemos, vamos a guardar en el diccionario donde tenemos los nombres de los blobs de todos los audios

for blob in blobs:
    
    actual_user = blob.name.split('/')[1].upper()
    
    if actual_user in used_users:
        diccionario_audios_used_users[actual_user].append(blob.name)
    
    elif actual_user in not_used_users:
        diccionario_audios_not_used_users[actual_user].append(blob.name)
        


In [ ]:
#Vamos a separar los audios en train y test.
#Recorremos el diccionario con todas las personas y audios, para cada persona, los primeros 3 audios que encontremos van a train y el resto a test

for persona in diccionario_audios_used_users.keys():
    
    audio_actual=0
    
    
    for audio in diccionario_audios_used_users.get(persona):
        
        audio_actual+=1
        
        
        if audio_actual<=3:
            
            diccionario_audios_train[persona].append(audio)
        
        else:
            
            diccionario_audios_test[persona].append(audio)
            

    audio_actual=0
        



In [ ]:
diccionario_audios_test

# 

# CARGA Y ALMACENAMIENTO DE DATOS

# 

# INFORMACIÓN DE SET DE DATOS

Se usó un corpus de audios reales grabados de diferentes dispositivos con sistema operativo Android.

Los hablantes utilizados son 8 personas de sexo masculino.

En total, se poseen 10 archivos de audio por persona, donde se utilizan 3 para entrenamiento, y el restante para verificación.

En total: 24 archivos de audio para entrenamiento (3 por persona), y 56 archivos de audio para testeo (7 por persona).

Notar que cada archivo de audio es el habla de una única persona.

En total, se procesaron 13804 Frames.

Además, se poseen 4 personas desconocidos para el modelo (impostores), para los cuales se cuentan con 10 audios de cada uno (40 en total).

Cabe destacar que para otorgarle un grado extra de complejidad a la identificación del hablante, 1 de los impostores es hijo de 1 de las personas presentes en el modelo.

Además, el modelo posee un padre y un hijo con los cuales se entreno. Y a su vez, otra de las personas desconocidas es hermano e hijo de las mencionadas

# 

# 

# PROCESADO DE AUDIOS DE ENTRENAMIENTO

In [ ]:
%%time
#Seteo la primer fila de los MFCC, luego la borro 
df_MFCC = np.zeros(cant_MFCC,)


#Seteo la primer fila de personas, luego la borro
df_personas = np.zeros(cant_personas_train,)


#Persona de la que leo los features
persona_actual = 0


#Auxiliar para saber la cantidad de frames totales para train
cant_frames_train = 0


#Recorro las carpetas de la región del dialecto, donde cada subcarpeta es una persona.
for persona in used_users:
    
    #Persona de la que leo los features
    persona_actual += 1

    
    #Contador de frames leidos de los archivos de la persona actual
    cant_frames_persona_actual = 0

    
    #Carpeta de los archivos de audio de cada persona
    for archivo_voz in diccionario_audios_train.get(persona):

    
        file = open('temp_audio.wav', 'wb+')
        bucket.get_blob(archivo_voz).download_to_file(file)
        file.close()
        
        #Extraigo los MFCC de un audio
        mfcc = loadMfccArchivo(file.name)
        
        
        #Actualmente en el array tengo n filas x m columnas, donde n es la cantidad de MFCC y m la cantidad de frames
        #Traspongo la matriz para quedarme con los features de un determinado frame, es decir n cantidad de MFCC
        #columnas x m cantidad de frames filas
        mfcc = np.transpose(mfcc)

        
        #Para agregar información temporal y fonética (velocidad, aceleración, entre otras) a los coeficientes
        #agrego los delta de cada feature y los concateno al array en formato de columnas
        mfcc = np.append(mfcc, librosa.feature.delta(mfcc), axis = 1)

        
        #apilo los features de los audios a un array
        df_MFCC = np.vstack((df_MFCC, mfcc))

        
        #Cuento los frames de la persona actual
        cant_frames_persona_actual+= mfcc.shape[0]


    #Ya lei todos los archivos de la persona
    #Paso a identificar la persona para almacenar los frames identificados
    for i in range(0, cant_frames_persona_actual):
   
        df_personas = agregarIdentificacionPersona(persona_actual, df_personas)
     
    
    print("Leídos {:d} frames de la persona {:d}".format(cant_frames_persona_actual, persona_actual))
    
    cant_frames_train += cant_frames_persona_actual


#Borro la primer fila de MFCC que eran todos 0
df_MFCC = np.delete(df_MFCC, (0), axis=0)


#Borro la primer fila de Personas que eran todos 0
df_personas = np.delete(df_personas, (0), axis=0)

        

In [ ]:
cant_frames_train

# 

# 

# PROCESADO DE AUDIOS DE TESTEO

In [ ]:
%%time
#Seteo la primer fila de los MFCC, luego la borro 
df_MFCC_test = np.zeros(cant_MFCC,)


#Seteo la primer fila de personas, luego la borro
df_personas_test = np.zeros(cant_personas_train,)


#Persona de la que leo los features
persona_actual = 0


#Auxiliar para saber la cantidad de frames totales para train
cant_frames_test = 0


#Recorro las carpetas de la región del dialecto, donde cada subcarpeta es una persona.
for persona in used_users:
    
    #Persona de la que leo los features
    persona_actual += 1

    
    #Contador de frames leidos de los archivos de la persona actual
    cant_frames_persona_actual = 0

    
    #Carpeta de los archivos de audio de cada persona
    for archivo_voz in diccionario_audios_test.get(persona):

    
        file = open('temp_audio.wav', 'wb+')
        bucket.get_blob(archivo_voz).download_to_file(file)
        file.close()
        
        #Extraigo los MFCC de un audio
        mfcc = loadMfccArchivo(file.name)

        
        #Actualmente en el array tengo n filas x m columnas, donde n es la cantidad de MFCC y m la cantidad de frames
        #Traspongo la matriz para quedarme con los features de un determinado frame, es decir n cantidad de MFCC
        #columnas x m cantidad de frames filas
        mfcc = np.transpose(mfcc)

        
        #Para agregar información temporal y fonética (velocidad, aceleración, entre otras) a los coeficientes
        #agrego los delta de cada feature y los concateno al array en formato de columnas
        mfcc = np.append(mfcc, librosa.feature.delta(mfcc), axis = 1)

        
        #apilo los features de los audios a un array
        df_MFCC_test = np.vstack((df_MFCC_test, mfcc))

        
        #Cuento los frames de la persona actual
        cant_frames_persona_actual+= mfcc.shape[0]


    #Ya lei todos los archivos de la persona
    #Paso a identificar la persona para almacenar los frames identificados
    for i in range(0, cant_frames_persona_actual):
   
        df_personas_test = agregarIdentificacionPersona(persona_actual, df_personas_test)
     
    
    print("Leídos {:d} frames de la persona {:d}".format(cant_frames_persona_actual, persona_actual))
    
    cant_frames_test += cant_frames_persona_actual


#Borro la primer fila de MFCC que eran todos 0
df_MFCC_test = np.delete(df_MFCC_test, (0), axis=0)


#Borro la primer fila de Personas que eran todos 0
df_personas_test = np.delete(df_personas_test, (0), axis=0)



In [ ]:
cant_frames_totales = cant_frames_test+cant_frames_train
print('cant frames test  ',cant_frames_test)
print('cant frames train ',cant_frames_train)
print('cant frames total ',cant_frames_totales)

print('distribucion train {:.2}'.format(cant_frames_train/cant_frames_totales))
print('distribucion test  {:.2}'.format(cant_frames_test/cant_frames_totales))

# 

# 

# ENTRENAMIENTO DE LA RED

In [ ]:
%%time
#CREO LA RED CON LAS CARACTERÍSTICAS MENCIONADAS
clasificador_voz = MLPClassifier(hidden_layer_sizes=(75, 75), activation='tanh', solver='adam', alpha=0.0001, 
                            batch_size='auto', learning_rate='adaptive', learning_rate_init=0.001, power_t=0.5, 
                            max_iter=100000, shuffle=True, random_state=None, tol=0.0001, verbose=True, warm_start=True, 
                            momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, 
                            beta_2=0.999, epsilon=1e-08, n_iter_no_change=150, max_fun=15000)



#Entreno la red
clasificador_voz.fit(df_MFCC, df_personas)

In [ ]:
guardar_output_como_pickle(user_to_train.replace(".","_")+".pickle", clasificador_voz)

In [ ]:
guardar_output_como_pickle(user_to_train.replace(".","_")+"_NAMES.pickle", used_users)

# 

# TESTEO DE LA RED

In [ ]:
#CANTIDAD DE FRAMES MAL CLASIFICADOS
cant_frames_mal_clasificados = 0

#CANTIDAD DE FRAMES BIEN CLASIFICADOS
cant_frames_bien_clasificados = 0



#PARA CADA FRAME DE TESTEO, VEO LA PREDICCIÓN DE LA RED
for i in range(0, df_MFCC_test.shape[0]):
    
    #Si la predicción del i-ésimo frame es igual al i-ésimo arreglo de personas que se identificaron
    #está bien clasificado
    if(np.array_equal(clasificador_voz.predict([df_MFCC_test[i]])[0], df_personas_test[i])):
        cant_frames_bien_clasificados += 1
    else:
        cant_frames_mal_clasificados += 1

        
print("BIEN CLASIFICADOS = ",cant_frames_bien_clasificados)

print("MAL CLASIFICADOS = ",cant_frames_mal_clasificados)


print('score TRAIN:', clasificador_voz.score(df_MFCC, df_personas)) 
print('score TEST:', clasificador_voz.score(df_MFCC_test, df_personas_test)) 

# 

### MATRIZ DE CONFUSIÓN DE FRAMES

In [ ]:
Y_test_pred = clasificador_voz.predict(df_MFCC_test)

fig = plt.figure(figsize=(15,6))

ax1 = fig.add_subplot(121)
skplt.metrics.plot_confusion_matrix(df_personas_test.argmax(axis=1), Y_test_pred.argmax(axis=1),
                                    title="Frames Confusion Matrix",
                                    cmap="Oranges",
                                    ax=ax1)

ax2 = fig.add_subplot(122)
skplt.metrics.plot_confusion_matrix(df_personas_test.argmax(axis=1), Y_test_pred.argmax(axis=1),
                                    normalize=True,
                                    title="Frames % Confusion Matrix",
                                    cmap="Purples",
                                    ax=ax2);

# 

In [ ]:
%%time
#CANTIDAD DE FRAMES CLASIFICADOS
cant_frames_mal_clasificados = 0
cant_frames_bien_clasificados = 0

#CANTIDAD DE PERSONAS CLASIFICADAS
cant_personas_mal_clasificadas = 0
cant_personas_bien_clasificadas = 0

frames_persona = np.zeros((8,))


#PARA CADA FRAME DE TESTEO, VEO LA PREDICCIÓN DE LA RED
for i in range(0, df_MFCC_test.shape[0]):
    
    
    #Si la predicción del i-ésimo frame es igual al i-ésimo arreglo de personas que se identificaron
    #está bien clasificado  
    
    if(np.array_equal(clasificador_voz.predict([df_MFCC_test[i]])[0], df_personas_test[i])):
        cant_frames_bien_clasificados += 1
        
    else:
        cant_frames_mal_clasificados += 1


        
#VEO LA PREDICCIÓN POR PERSONAS DE LA RED
for persona in range(0, cant_personas_train):

    #RECORRO LOS FRAMES DE TESTEO PARA VALIDAR LA RED
    for frame in range(0, df_MFCC_test.shape[0]):
  
        #ME QUEDO CON LOS FRAMES QUE IDENTIFIQUE QUE SEAN DE LA PERSONA
        #ESTOS SON LOS FRAMES IDENTIFICADOS QUE USO PARA TESTEO (VALIDACION)
        if(df_personas_test[frame][persona] == 1):
      
            #RECORRO DE 0 a 7, 8 PERSONAS. PARA SABER SI EL OUTPUT QUE ME DIO LA RED
            #ES EL DE LA PERSONA QUE YO NECESITO
            for j in range(0, cant_personas_train):
        
                #SI ES EL DE ESA PERSONA, SETEO QUE LE IDENTIFIQUE UN FRAME
                if(clasificador_voz.predict([df_MFCC_test[frame]])[0][j] == 1):
                    frames_persona[j] += 1
  
    #YA RECORRI TODOS LOS FRAMES DE TESTEO, AHORA ME FIJO CON QUE PERSONA SE
    #IDENTIFICARON LA MAYOR CANTIDAD DE FRAMES

    persona_mas_frames = 0
    cant_frames_mayor_de_persona_identificada = 0

    #VEO EL VECTOR DONDE ME GUARDE LA CANTIDAD DE FRAMES QUE IDENTIFIQUE PARA
    #CADA PERSONA
    for persona_identificacion in range(0,cant_personas_train):

        if(frames_persona[persona_identificacion] > cant_frames_mayor_de_persona_identificada):
            cant_frames_mayor_de_persona_identificada = frames_persona[persona_identificacion]
            persona_mas_frames = persona_identificacion

    #YA TENGO QUE PERSONA FUE CON LA QUE MAS FRAMES SE IDENTIFICARON VEO SI ES LA
    #QUE TENIA QUE SER

    if(persona_mas_frames == persona):
        cant_personas_bien_clasificadas += 1
    else:
        cant_personas_mal_clasificadas += 1



    print("ERA LA PERSONA ",persona," IDENTIFIQUE :",frames_persona)
    frames_persona = np.zeros((8,))




print("FRAMES TEST BIEN CLASIFICADOS = ",cant_frames_bien_clasificados)
print("FRAMES TEST MAL CLASIFICADOS = ",cant_frames_mal_clasificados)
print("PERSONAS TEST BIEN CLASIFICADAS = ",cant_personas_bien_clasificadas)
print("PERSONAS TEST MAL CLASIFICADAS = ",cant_personas_mal_clasificadas)


print('score FRAMES TRAIN:', clasificador_voz.score(df_MFCC, df_personas)) 
print('score FRAMES TEST:', clasificador_voz.score(df_MFCC_test, df_personas_test)) 
print('score PERSONAS TEST:', cant_personas_train/cant_personas_bien_clasificadas)

In [ ]:
cant_frames_totales = cant_frames_test+cant_frames_train
print('cant frames test  ',cant_frames_test)
print('cant frames train ',cant_frames_train)
print('cant frames total ',cant_frames_totales)

print('distribucion train {:.2%}'.format(cant_frames_train/cant_frames_totales))
print('distribucion test  {:.2%}'.format(cant_frames_test/cant_frames_totales))

# 

# ACA YA FINALIZA EL MODELO

# 

In [ ]:
%%time
#Testeo de cada audio que deje para test


filas_dataframe = [[None,None,None,None]]

#Persona de la que leo los features
persona_actual = 0


#Auxiliar para saber la cantidad de frames totales para train
cant_frames_test = 0


#CANTIDAD TOTAL DE FRAMES CLASIFICADOS
cant_total_frames_mal_clasificados = 0
cant_total_frames_bien_clasificados = 0

#CANTIDAD DE AUDIOS CLASIFICADOS
cant_audios_mal_clasificados = 0
cant_audios_bien_clasificados = 0


#Recorro las carpetas de la región del dialecto, donde cada subcarpeta es una persona.
for persona in used_users:

    
    #Persona de la que leo los features
    persona_actual += 1

    
    #Contador de frames leidos de los archivos de la persona actual
    cant_frames_persona_actual = 0

    
    #Carpeta de los archivos de audio de cada persona
    for archivo_voz in diccionario_audios_test.get(persona):

    
        file = open('temp_audio.wav', 'wb+')
        bucket.get_blob(archivo_voz).download_to_file(file)
        file.close()
        
        #Extraigo los MFCC de un audio
        mfcc = loadMfccArchivo(file.name)
        
        print("PERSONA ",persona_actual, " --------------------- ARCHIVO ",archivo_voz)
    

        
        #Actualmente en el array tengo n filas x m columnas, donde n es la cantidad de MFCC y m la cantidad de frames
        #Traspongo la matriz para quedarme con los features de un determinado frame, es decir n cantidad de MFCC
        #columnas x m cantidad de frames filas
        mfcc = np.transpose(mfcc)
        
        
        #Para agregar información temporal y fonética (velocidad, aceleración, entre otras) a los coeficientes
        #agrego los delta de cada feature y los concateno al array en formato de columnas
        mfcc = np.append(mfcc, librosa.feature.delta(mfcc), axis = 1)
        
        
        
        #Contador de frames leidos de los archivos de la persona actual
        cant_frames_audio_actual = mfcc.shape[0]
        

        
        #CANTIDAD DE FRAMES CLASIFICADOS DEL AUDIO ACTUAL
        cant_frames_audio_actual_mal_clasificados = 0
        cant_frames_audio_actual_bien_clasificados = 0
        
        
        #ARRAY PARA ASIGNAR LAS PREDICCIONES DE CADA FRAME DEL AUDIO ACTUAL, A PERSONAS
        predicciones_frames_personas = np.zeros((cant_personas_train,))

        
        #Recorro todos los frames del audio
        for frame in mfcc:
            
            prediccion = clasificador_voz.predict([frame])[0]
            
            if prediccion[persona_actual-1] == 1:
                cant_frames_audio_actual_bien_clasificados+=1
            else:
                cant_frames_audio_actual_mal_clasificados+=1
         
            #RECORRO EL FRAME PARA SABER A QUE PERSONA CORRESPONDE LA PREDICCION
            for i in range(0, prediccion.shape[0]):
        
                #SI ES EL DE ESA PERSONA, SETEO QUE LE IDENTIFIQUE UN FRAME, Y NO RECORRO MÁS
                if(prediccion[i] == 1):
                    predicciones_frames_personas[i] += 1
                    break
                 
                
        #PERSONA IDENTIFICADA. INDICE MAXIMO +1 
        persona_identificada = predicciones_frames_personas.argmax()+1
        
        
        if(persona_identificada == persona_actual):
            cant_audios_bien_clasificados += 1
        else:
            cant_audios_mal_clasificados += 1

        
        
        cant_total_frames_mal_clasificados += cant_frames_audio_actual_mal_clasificados
        cant_total_frames_bien_clasificados += cant_frames_audio_actual_bien_clasificados
            
        filas_dataframe.append([archivo_voz, persona_actual, cant_frames_audio_actual, predicciones_frames_personas])



In [ ]:
print("Audios BIEN clasificados = ",cant_audios_bien_clasificados)
print("Audios MAL clasificados  = ",cant_audios_mal_clasificados)
print("Frames BIEN clasificados = ",cant_total_frames_bien_clasificados)
print("Frames MAL clasificados  = ",cant_total_frames_mal_clasificados)

## Creamos el DF del resultado

In [ ]:
columns = ['Audio', 'Persona del audio', 'Cantidad de frames', 'Predicciones en frames']


df_resultado = pd.DataFrame(filas_dataframe, columns=columns)

df_resultado.drop(0, inplace=True)

df_resultado

# 

### MATRIZ DE CONFUSIÓN DE LOS AUDIOS

In [ ]:
# from sklearn.metrics import classification_report
# from sklearn.metrics import confusion_matrix

# print(classification_report(Y_test_real_audio, Y_test_pred_audio, digits=2))


In [ ]:
Y_test_pred_audio = []
Y_test_real_audio = df_resultado['Persona del audio'].to_numpy()

#Tomo la columna predicción en frames de cada audio y sumo 1 al argmax para que las personas sean de 1 a N
for pred in df_resultado['Predicciones en frames']:
    Y_test_pred_audio.append(pred.argmax()+1)

    
    
    
fig = plt.figure(figsize=(15,6))

ax1 = fig.add_subplot(121)
skplt.metrics.plot_confusion_matrix(Y_test_real_audio, Y_test_pred_audio,
                                    title="Audios Confusion Matrix",
                                    cmap="Oranges",
                                    ax=ax1)

ax2 = fig.add_subplot(122)
skplt.metrics.plot_confusion_matrix(Y_test_real_audio, Y_test_pred_audio,
                                    normalize=True,
                                    title="Audios % Confusion Matrix",
                                    cmap="Purples",
                                    ax=ax2);


In [ ]:
y_actu = pd.Series(Y_test_real_audio, name='Actual')
y_pred = pd.Series(Y_test_pred_audio, name='Predicted')
df_confusion = pd.crosstab(y_actu, y_pred)
print("CONFUSION MATRIX FOR TRAIN AUDIOS\n\n")
print(df_confusion)

In [ ]:
audios_accuracy = metrics.accuracy_score(Y_test_real_audio, Y_test_pred_audio)
audios_precision = metrics.precision_score(Y_test_real_audio, Y_test_pred_audio, average='weighted')


print("AUDIOS ACCURACY TEST = {:.2%}".format(audios_accuracy))
print("AUDIOS PRECISION TEST = {:.2%}".format(audios_precision))

# 

## CONCLUSIONES INICIALES

A nivel audios hay un excelente resultado.

Notamos una gran diferencia entre la asignación de frames de un audio, entre las dos primeras personas con mayor asignación de frames. Es decir, asigna muchos más frames a la primera persona que a la segunda.

A la persona identificada en primer lugar, le asigna una gran cantidad de frames del audio respecto al total del mismo.




#### 

A partir de estas conclusiones es que vamos a determinar un umbral de diferencia entre la asignación de frames del total del audio y de diferencia entre la primera persona y la segunda para poder filtrar los "impostores"

# 

In [ ]:
def calcular_desvio_entre_frames_asignados_a_personas(frames_asignados_a_personas):
    
    persona_max_asignaciones = 1
    frames_persona_max_asignaciones = frames_asignados_a_personas[0]
    
    persona_segunda_max_asignaciones = 0
    frames_persona_segunda_max_asignaciones = 0
    
    
    for i in range(1,len(frames_asignados_a_personas)):
        
        if (frames_asignados_a_personas[i]>frames_persona_max_asignaciones):
            
            persona_segunda_max_asignaciones = persona_max_asignaciones
            frames_persona_segunda_max_asignaciones = frames_persona_max_asignaciones
            
            persona_max_asignaciones = i+1
            frames_persona_max_asignaciones = frames_asignados_a_personas[i]
            
            
            
        elif (frames_asignados_a_personas[i]>frames_persona_segunda_max_asignaciones):
            
            persona_segunda_max_asignaciones = i+1
            frames_persona_segunda_max_asignaciones = frames_asignados_a_personas[i] 
                       
     
    desvio_frames_primer_y_segunda_persona = (frames_persona_max_asignaciones/frames_persona_segunda_max_asignaciones)-1
    
    
    return desvio_frames_primer_y_segunda_persona
    

In [ ]:
columns = ['Audio', 'Persona del audio', 'Cantidad de frames', 'Predicciones en frames']


df_resultado = pd.DataFrame(filas_dataframe, columns=columns)


df_resultado.drop(0, inplace=True)


df_resultado['Persona del audio'] = df_resultado['Persona del audio'].astype(int)


df_resultado['Frames bien clasificados'] = df_resultado.apply(lambda row : 
                                                              row['Predicciones en frames'][row['Persona del audio']-1]
                                                              .astype(int), axis=1)


df_resultado['Frames mal clasificados'] = df_resultado.apply(lambda row : 
                                                              (row['Predicciones en frames'].sum() -
                                                               row['Frames bien clasificados'])
                                                              .astype(int), axis=1)


df_resultado['Frames no clasificados'] = df_resultado.apply(lambda row : 
                                                              (row['Cantidad de frames'] - 
                                                               row['Predicciones en frames'].sum())
                                                              .astype(int), axis=1)


df_resultado['Porcentaje no clasificados'] = df_resultado.apply(lambda row : 
                                                              math.ceil(row['Frames no clasificados'] / 
                                                               row['Cantidad de frames']*100)
                                                              , axis=1)


df_resultado['Persona predicha'] = df_resultado['Predicciones en frames'].apply(lambda x : x.argmax()+1)


df_resultado['Prediccion correcta'] = df_resultado['Persona predicha'] == df_resultado['Persona del audio']


df_resultado['Persona segunda max frames'] = (df_resultado['Predicciones en frames']
                                              .apply(lambda x : np.argwhere(x==np.sort(x)[-2])[0][0]+1))


df_resultado['Frames persona segunda max frames'] = df_resultado.apply(lambda row : 
                                                            (row['Predicciones en frames'][row['Persona segunda max frames']-1])
                                                              .astype(int)
                                                              , axis=1)


df_resultado['Frames Persona Max Frames'] = df_resultado.apply(lambda row: int(row['Predicciones en frames']
                                                               [row['Persona predicha']-1]), axis=1)


df_resultado['Porcentaje diferencia primera persona con segunda'] = (((df_resultado['Frames Persona Max Frames'] /
                                                                     df_resultado['Frames persona segunda max frames'])-1)*100).apply(lambda x: math.ceil(x))


df_resultado['Porcentaje max sobre total'] = (df_resultado['Frames Persona Max Frames'] / df_resultado['Cantidad de frames'])*100

df_resultado['Porcentaje max sobre total sin blancos'] = (df_resultado['Frames Persona Max Frames'] / (df_resultado['Cantidad de frames']-df_resultado['Frames no clasificados']))*100


df_resultado['Desvio'] = df_resultado['Predicciones en frames'].apply(lambda x : calcular_desvio_entre_frames_asignados_a_personas(x))


df_resultado


# 

# Analisis diferencias primera persona identificada vs segunda

Para hacer un poco más riguroso el análisis, tomemos el promedio de los 5 porcentajes más bajos de diferencia, ya que solemos tener una diferencia MUY amplia

In [ ]:
df_resultado.sort_values(by='Porcentaje diferencia primera persona con segunda'
                        )['Porcentaje diferencia primera persona con segunda'][:5].values.mean()

Dado este resultado, podemos establecer un umbral de "validación" del 50% para hacer una validación más rigurosa y poder filtrar posibles impostores

# 

## Análisis porcentajes del total del audio asignado a la persona identificada

Para hacer un poco más riguroso el análisis, tomemos el promedio de los 5 porcentajes más bajos de asignación del total de frames del audio

In [ ]:
df_resultado.sort_values(by='Porcentaje max sobre total'
                        )['Porcentaje max sobre total'][:5].values.mean()


Dado este resultado, podemos establecer un umbral de "validación" del 30% para hacer una validación más rigurosa y poder filtrar posibles impostores

# 

In [ ]:
def validar_prediccion(predicciones_frames, porcentaje_sobre_total):
    desvio = calcular_desvio_entre_frames_asignados_a_personas(predicciones_frames)
    
    validacion_desvio = (1 if desvio>=.5 else (.5 if desvio>=.25 else 0))
    validacion_porcentaje = (1 if np.ceil(porcentaje_sobre_total)>=40 else 0)
    
    return min(validacion_desvio, validacion_porcentaje)


# 

## Probemos resultados con este validador

In [ ]:
df_resultado['Validar Asignacion'] = df_resultado.apply(lambda row: 
                                                        validar_prediccion(row['Predicciones en frames'], 
                                                                              row['Porcentaje max sobre total'])
                                                        , axis=1)


df_resultado['prediccion_validada'] = df_resultado.apply(lambda row: row['Persona predicha'] if 
                                                         row['Validar Asignacion'] else -1, axis=1)


df_resultado

In [ ]:
df_resultado[df_resultado['prediccion_validada']==-1]

In [ ]:
# la columna predicción en frames de cada audio y sumo 1 al argmax para que las personas sean de 1 a N

    
    
    
fig = plt.figure(figsize=(15,6))

ax1 = fig.add_subplot(121)
skplt.metrics.plot_confusion_matrix(df_resultado['Persona del audio'].to_numpy(), 
                                    df_resultado['prediccion_validada'].to_numpy(),
                                    title="Audios Confusion Matrix",
                                    cmap="Oranges",
                                    ax=ax1)

ax2 = fig.add_subplot(122)
skplt.metrics.plot_confusion_matrix(df_resultado['Persona del audio'].to_numpy(), 
                                    df_resultado['prediccion_validada'].to_numpy(),
                                    normalize=True,
                                    title="Audios % Confusion Matrix",
                                    cmap="Blues",
                                    ax=ax2);


In [ ]:
y_actu = pd.Series(df_resultado['Persona del audio'], name='Actual')
y_pred = pd.Series(df_resultado['prediccion_validada'], name='Predicted')
df_confusion = pd.crosstab(y_actu, y_pred)
print("CONFUSION MATRIX FOR TEST AUDIOS\n\n")
print(df_confusion)

In [ ]:
print(print("Predicciones de audios de personas conocidas por la red con validaciones \n\n"))

print(df_resultado[['Persona del audio','Prediccion correcta','Validar Asignacion']].groupby(['Prediccion correcta','Validar Asignacion']).count().reset_index())

In [ ]:
audios_accuracy_validador = metrics.accuracy_score(df_resultado['Persona del audio'].to_numpy(),
                                                   df_resultado['prediccion_validada'].to_numpy())

audios_precision_validador = metrics.precision_score(df_resultado['Persona del audio'].to_numpy(), 
                                                     df_resultado['prediccion_validada'].to_numpy(), average='weighted')



print("AUDIOS ACCURACY = {:.2%}".format(audios_accuracy_validador))
print("AUDIOS PRECISION = {:.2%}".format(audios_precision_validador))

Si bien se reduce el accuracy , logramos un 100% de precisión. Es decir, que nunca el clasificador considerará válida una predicción, si no está realmente seguro (no hay FP)

# 

In [ ]:
%%time
#Testeo de cada audio que deje para test


filas_dataframe_desconocido = [[None,None,None,None]]


#Auxiliar para saber la cantidad de frames totales para train
cant_frames_persona_desconocida = 0



#Recorro las carpetas de la región del dialecto, donde cada subcarpeta es una persona.
for persona in not_used_users:

    print(persona)
    #Carpeta de los archivos de audio de cada persona
    for archivo_voz in diccionario_audios_not_used_users.get(persona):
        
        print("PERSONA ",persona, " --------------------- ARCHIVO ",archivo_voz)
    
        file = open('temp_audio.wav', 'wb+')
        bucket.get_blob(archivo_voz).download_to_file(file)
        file.close()
        
        #Extraigo los MFCC de un audio
        mfcc = loadMfccArchivo(file.name)
        
        
        #Actualmente en el array tengo n filas x m columnas, donde n es la cantidad de MFCC y m la cantidad de frames
        #Traspongo la matriz para quedarme con los features de un determinado frame, es decir n cantidad de MFCC
        #columnas x m cantidad de frames filas
        mfcc = np.transpose(mfcc)
        
        
        #Para agregar información temporal y fonética (velocidad, aceleración, entre otras) a los coeficientes
        #agrego los delta de cada feature y los concateno al array en formato de columnas
        mfcc = np.append(mfcc, librosa.feature.delta(mfcc), axis = 1)
        
        
        
        
        
        
        
        
        #Contador de frames leidos de los archivos de la persona actual
        cant_frames_audio_actual = mfcc.shape[0]
        
        cant_frames_persona_desconocida+=cant_frames_audio_actual
        
    
        
        #CANTIDAD DE FRAMES CLASIFICADOS DEL AUDIO ACTUAL
        cant_frames_audio_actual_mal_clasificados = 0
        
        
        #ARRAY PARA ASIGNAR LAS PREDICCIONES DE CADA FRAME DEL AUDIO ACTUAL, A PERSONAS
        predicciones_frames_personas = np.zeros((cant_personas_train,))

        
        #Recorro todos los frames del audio
        for frame in mfcc:
            
            prediccion = clasificador_voz.predict([frame])[0]
            
            cant_frames_audio_actual_mal_clasificados+=1
         
            #RECORRO EL FRAME PARA SABER A QUE PERSONA CORRESPONDE LA PREDICCION
            for i in range(0, prediccion.shape[0]):
        
                #SI ES EL DE ESA PERSONA, SETEO QUE LE IDENTIFIQUE UN FRAME, Y NO RECORRO MÁS
                if(prediccion[i] == 1):
                    predicciones_frames_personas[i] += 1
                    break
                 
                
        #PERSONA IDENTIFICADA. INDICE MAXIMO +1 
        persona_identificada = predicciones_frames_personas.argmax()+1
        

            
        filas_dataframe_desconocido.append([persona+"\\"+archivo_voz, 8, cant_frames_audio_actual, predicciones_frames_personas])

In [ ]:
columns = ['Audio', 'Persona del audio', 'Cantidad de frames', 'Predicciones en frames']


df_resultado_desconocido = pd.DataFrame(filas_dataframe_desconocido, columns=columns)


df_resultado_desconocido.drop(0, inplace=True)

df_resultado_desconocido['Nombre_persona'] = df_resultado_desconocido['Audio'].apply(lambda x: x.split('\\')[0])

df_resultado_desconocido['Persona del audio'] = -1



df_resultado_desconocido['Frames bien clasificados'] = 0


df_resultado_desconocido['Frames mal clasificados'] = df_resultado_desconocido.apply(lambda row: row['Predicciones en frames'].sum(), axis=1)


df_resultado_desconocido['Frames no clasificados'] = df_resultado_desconocido.apply(lambda row : 
                                                              (row['Cantidad de frames'] - 
                                                               row['Predicciones en frames'].sum())
                                                              .astype(int), axis=1)


df_resultado_desconocido['Porcentaje no clasificados'] = df_resultado_desconocido.apply(lambda row : 
                                                              math.ceil(row['Frames no clasificados'] / 
                                                               row['Cantidad de frames']*100)
                                                              , axis=1)


df_resultado_desconocido['Persona predicha'] = df_resultado_desconocido['Predicciones en frames'].apply(lambda x : x.argmax()+1)


df_resultado_desconocido['Prediccion correcta'] = df_resultado_desconocido['Persona predicha'] == df_resultado_desconocido['Persona del audio']


df_resultado_desconocido['Persona segunda max frames'] = (df_resultado_desconocido['Predicciones en frames']
                                              .apply(lambda x : np.argwhere(x==np.sort(x)[-2])[0][0]+1))


df_resultado_desconocido['Frames persona segunda max frames'] = df_resultado_desconocido.apply(lambda row : 
                                                            (row['Predicciones en frames'][row['Persona segunda max frames']-1])
                                                              .astype(int)
                                                              , axis=1)


df_resultado_desconocido['Frames Persona Max Frames'] = df_resultado_desconocido.apply(lambda row: int(row['Predicciones en frames']
                                                               [row['Persona predicha']-1]), axis=1)


df_resultado_desconocido['Porcentaje diferencia primera persona con segunda'] = (((df_resultado_desconocido['Frames Persona Max Frames'] /
                                                                     df_resultado_desconocido['Frames persona segunda max frames'])-1)*100).apply(lambda x: math.ceil(x))




df_resultado_desconocido['Porcentaje max sobre total'] = (df_resultado_desconocido['Frames Persona Max Frames'] / df_resultado_desconocido['Cantidad de frames'])*100

df_resultado_desconocido['Porcentaje max sobre total sin blancos'] = (df_resultado_desconocido['Frames Persona Max Frames'] / (df_resultado_desconocido['Cantidad de frames']-df_resultado_desconocido['Frames no clasificados']))*100

df_resultado_desconocido['Validar Asignacion'] = df_resultado_desconocido.apply(lambda row: validar_prediccion(row['Predicciones en frames'], row['Porcentaje max sobre total']), axis=1)



df_resultado_desconocido['Desvio'] = df_resultado_desconocido['Predicciones en frames'].apply(lambda x : calcular_desvio_entre_frames_asignados_a_personas(x))


df_resultado_desconocido


In [ ]:
persona_actual = 0
filas_personas_nombres = [[None,None]]

for persona in used_users:

    
    #Persona de la que leo los features
    persona_actual += 1

    filas_personas_nombres.append([persona_actual, persona])
    
personas_nombres = pd.DataFrame(filas_personas_nombres, columns=['Persona predicha', 'Nombre_persona_predicha']).dropna()    
personas_nombres

print(personas_nombres)

# 

## Analicemos resultados de los impostores

In [ ]:
df_resultado_desconocido['prediccion_validada'] = df_resultado_desconocido.apply(lambda row: row['Persona predicha'] if 
                                                         row['Validar Asignacion'] else -1, axis=1)
    
    
    
fig = plt.figure(figsize=(15,6))

ax1 = fig.add_subplot(121)
skplt.metrics.plot_confusion_matrix(df_resultado_desconocido['Persona del audio'].to_numpy(), 
                                    df_resultado_desconocido['prediccion_validada'].to_numpy(),
                                    title="Audios Confusion Matrix",
                                    cmap="Oranges",
                                    ax=ax1)

ax2 = fig.add_subplot(122)
skplt.metrics.plot_confusion_matrix(df_resultado_desconocido['Persona del audio'].to_numpy(), 
                                    df_resultado_desconocido['prediccion_validada'].to_numpy(),
                                    normalize=True,
                                    title="Audios % Confusion Matrix",
                                    cmap="Blues",
                                    ax=ax2);


In [ ]:
y_actu = pd.Series(df_resultado_desconocido['Persona del audio'], name='Actual')
y_pred = pd.Series(df_resultado_desconocido['prediccion_validada'], name='Predicted')
df_confusion = pd.crosstab(y_actu, y_pred)
print("CONFUSION MATRIX FOR IMPOSTORS AUDIOS\n\n")
print(df_confusion)

In [ ]:
print("Predicciones de audios de impostores\n\n")
print(df_resultado_desconocido[['Prediccion correcta', 'Validar Asignacion', 'Audio']].groupby(['Prediccion correcta', 'Validar Asignacion']).count().reset_index())

In [ ]:
df_resultado_desconocido[['Prediccion correcta', 'Validar Asignacion', 'Audio']].groupby(['Prediccion correcta', 'Validar Asignacion']).count().reset_index()

In [ ]:

audios_accuracy_validador = metrics.accuracy_score(df_resultado_desconocido['Persona del audio'].to_numpy(),
                                                   df_resultado_desconocido['prediccion_validada'].to_numpy())

audios_precision_validador = metrics.precision_score(df_resultado_desconocido['Persona del audio'].to_numpy(), 
                                                     df_resultado_desconocido['prediccion_validada'].to_numpy(), average='weighted')



print("AUDIOS IMPOSTORES ACCURACY = {:.2%}".format(audios_accuracy_validador))
print("AUDIOS IMPOSTORES PRECISION = {:.2%}".format(audios_precision_validador))

# 

In [ ]:
print('score FRAMES TRAIN:', clasificador_voz.score(df_MFCC, df_personas)) 
print('score FRAMES TEST:', clasificador_voz.score(df_MFCC_test, df_personas_test)) 

In [ ]:
cant_frames_totales = cant_frames_test+cant_frames_train
print('cant frames test  ',cant_frames_test)
print('cant frames train ',cant_frames_train)
print('cant frames total ',cant_frames_totales)

print('distribucion train {:.2}'.format(cant_frames_train/cant_frames_totales))
print('distribucion test  {:.2}'.format(cant_frames_test/cant_frames_totales))

In [ ]:
print("BIEN CLASIFICADOS = ",cant_frames_bien_clasificados)

print("MAL CLASIFICADOS = ",cant_frames_mal_clasificados)


print('score TRAIN:', clasificador_voz.score(df_MFCC, df_personas)) 
print('score TEST:', clasificador_voz.score(df_MFCC_test, df_personas_test)) 

In [ ]:
audios_accuracy = metrics.accuracy_score(Y_test_real_audio, Y_test_pred_audio)
audios_precision = metrics.precision_score(Y_test_real_audio, Y_test_pred_audio, average='weighted')


print("AUDIOS ACCURACY TEST = {:.2%}".format(audios_accuracy))
print("AUDIOS PRECISION TEST = {:.2%}".format(audios_precision))